In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from numpy.random import uniform, normal

In [ ]:
def sample_k_slopes(num, thres, mean1, dev1, mean2, dev2):
    probabilities = uniform(num)
    
    dist = [normal(mean1, dev1) if p < thres else normal(mean2, dev2) for p in probabilities]
    return dist

def sample_v_slopes(num, mean, dev):
    return normal(mean, dev, num)

In [ ]:
np.random.seed(0)
plt.rcParams['figure.figsize'] = (10, 8)

In [ ]:
# independent global constants

# patient constants
k_slope = 0.003125
k_int = 0.01
cl_cr = 50
v_slope = 0.2806
bw = 70

# error constants
measurement_error = 0
measurement_timing_error = 0
dosage_error = 0
dosage_timing_error = 0

# therapy constants
days = 10
dosage_period = 12
dosage_length = 1
peak = 7
trough = 1.5
step_size = 0.1
measurement_times = [1, 11, 73, 83, 145, 155]

# prior constants
size = 9
p = 0.2
k_dev = 0.001
v_dev = 0.05

In [ ]:
# dependent global constants

# patient constants
k_el = k_int + k_slope * cl_cr
v_d = v_slope * bw

# therapy constants
duration = days * 24
num_of_doses = int(duration/dosage_period)
dosage_times = [ dosage_period*i for i in range(num_of_doses) ]

# prior constants
mean1 = 4/3*k_slope
mean2 = 2/3*k_slope

k_slopes = sample_k_slopes(size, p, mean1, k_dev, mean2, k_dev)
v_slopes = sample_v_slopes(size, v_slope, v_dev)

In [ ]:
class Simulator:
    def __init__(self):
        self.measurements = []
        self.prior = self.initializePrior()
        self.dose = self.optimalDose()
        
    def initializePrior():
        global size, k_slopes, v_slopes
        
        w = 1/size**2
        prior = w * np.ones((size, size))
        
        return prior
    
    def optimalDose(lo=50, hi=200):
        global k_slopes, v_slopes